> **Team5_IE212.O13.CNCL**
* Member: 
1.  Nguyễn Xuân Anh Vũ
2.  Võ Chí Trường
3. Võ Huỳnh Anh Vũ
4. Trần Thanh Phong

In [375]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uit-diabetic-patients-re-admission-prediction/IDs_mapping.csv
/kaggle/input/uit-diabetic-patients-re-admission-prediction/test_diabetic_data.csv
/kaggle/input/uit-diabetic-patients-re-admission-prediction/train_diabetic_data.csv
/kaggle/input/uit-diabetic-patients-re-admission-prediction/2014 beata strack et al.pdf
/kaggle/input/uit-diabetic-patients-re-admission-prediction/2016 bhuvan m s et al.pdf
/kaggle/input/uit-diabetic-patients-re-admission-prediction/sample_solution.csv


In [376]:
!pip install pyspark

In [377]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from pyspark.sql.functions import col, when, mean, udf,lit
import seaborn as sns
import statsmodels.api as sm
import pyspark

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

from pyspark.ml.feature import StringIndexer, VectorIndexer, StringIndexerModel, IndexToString, OneHotEncoder
from pyspark.sql import SparkSession

from pyspark.sql.types import StringType
spark = SparkSession.builder.appName("YourAppName").config("spark.executor.memory", "4g").getOrCreate()

In [378]:
df = spark.read.format("csv").load("/kaggle/input/uit-diabetic-patients-re-admission-prediction/train_diabetic_data.csv",header = 'True',inferSchema='True')

In [379]:
test = spark.read.format("csv").load("/kaggle/input/uit-diabetic-patients-re-admission-prediction/test_diabetic_data.csv",header = 'True',inferSchema='True')

In [380]:
for col_name in df.columns:
    null_count = df.filter(col(col_name) == '?').count()
    print(f"Number of null values in {col_name}: {null_count}")

Number of null values in encounter_id: 0
Number of null values in patient_nbr: 0
Number of null values in race: 1878
Number of null values in gender: 0
Number of null values in age: 0
Number of null values in weight: 68671
Number of null values in admission_type_id: 0
Number of null values in discharge_disposition_id: 0
Number of null values in admission_source_id: 0
Number of null values in time_in_hospital: 0
Number of null values in payer_code: 30085
Number of null values in medical_specialty: 34525
Number of null values in num_lab_procedures: 0
Number of null values in num_procedures: 0
Number of null values in num_medications: 0
Number of null values in number_outpatient: 0
Number of null values in number_emergency: 0
Number of null values in number_inpatient: 0
Number of null values in diag_1: 17
Number of null values in diag_2: 290
Number of null values in diag_3: 1146
Number of null values in number_diagnoses: 0
Number of null values in max_glu_serum: 0
Number of null values in

In [381]:
df = df.drop('encounter_id', 'patient_nbr')

In [382]:
df = df.withColumn('race', when(col('race') == '?', None).otherwise(col('race')))

In [383]:
test = test.drop('encounter_id', 'patient_nbr')

In [384]:
test = test.withColumn('race', when(col('race') == '?', None).otherwise(col('race')))

In [385]:
df = df.withColumn('gender', when(col('gender') == 'Unknown/Invalid', 0).otherwise(col('gender')))


In [386]:
test = test.withColumn('gender', when(col('gender') == 'Unknown/Invalid', 0).otherwise(col('gender')))


In [387]:
df = df.withColumn('gender', when(col('gender') == 'Female', 0).when(col('gender') == 'Male', 1).otherwise(col('gender')))

In [388]:
test = test.withColumn('gender', when(col('gender') == 'Female', 0).when(col('gender') == 'Male', 1).otherwise(col('gender')))

In [389]:
df = df.withColumn(
    'age',
    when(col('age') == '[70-80)', 75)
    .when(col('age') == '[60-70)', 65)
    .when(col('age') == '[80-90)', 85)
    .when(col('age') == '[50-60)', 55)
    .when(col('age') == '[40-50)', 45)
    .when(col('age') == '[30-40)', 35)
    .when(col('age') == '[90-100)', 95)
    .when(col('age') == '[20-30)', 25)
    .when(col('age') == '[10-20)', 15)
    .when(col('age') == '[0-10)', 5)
    .otherwise(col('age'))
)
df = df.withColumn('age', col('age').cast('int'))
# Show the shape of the resulting DataFrame
print("DataFrame shape after processing: ({}, {})".format(df.count(), len(df.columns)))


DataFrame shape after processing: (71518, 48)


In [390]:
test = test.withColumn(
    'age',
    when(col('age') == '[70-80)', 75)
    .when(col('age') == '[60-70)', 65)
    .when(col('age') == '[80-90)', 85)
    .when(col('age') == '[50-60)', 55)
    .when(col('age') == '[40-50)', 45)
    .when(col('age') == '[30-40)', 35)
    .when(col('age') == '[90-100)', 95)
    .when(col('age') == '[20-30)', 25)
    .when(col('age') == '[10-20)', 15)
    .when(col('age') == '[0-10)', 5)
    .otherwise(col('age'))
)
test = test.withColumn('age', col('age').cast('int'))
# Show the shape of the resulting DataFrame
print("DataFrame shape after processing: ({}, {})".format(test.count(), len(test.columns)))


DataFrame shape after processing: (30248, 48)


In [391]:
df = df.drop('weight')

In [392]:
test = test.drop('weight')

In [393]:
df = df.withColumn(
    'admission_type_id',
    when(col('admission_type_id').isin([1, 2, 7]), 'Emergency')
    .when(col('admission_type_id').isin([5, 6, 8]), 'Not Available')
    .when(col('admission_type_id').isin([3]), 'Elective')
    .when(col('admission_type_id').isin([4]), 'Newborn')
    .otherwise('None')
)
#data = data.filter(col('admission_type_id') != 'Newborn')
df = df.withColumn(
    'discharge_disposition_id',
     when(col('discharge_disposition_id').isin([1]), 'Discharged to home')
    .when(col('discharge_disposition_id').isin([2, 3, 4, 5, 10, 16, 22, 23, 24, 30, 27, 28, 29]), 'Transferred to another medical facility')
    .when(col('discharge_disposition_id').isin([7]), 'Left AMA(Against Medical Advice.)')
    .when(col('discharge_disposition_id').isin([6, 8]), 'Discharged to home with home health service')
    .when(col('discharge_disposition_id').isin([9, 12, 15, 17]), 'Still patient/referred to this institution')
    .when(col('discharge_disposition_id').isin([11, 19, 20, 21]), 'Expired')
    .when(col('discharge_disposition_id').isin([18, 25, 26]), 'Not Available')
    .when(col('discharge_disposition_id').isin([13, 14]), 'Hospice')
    .when(col('discharge_disposition_id').isin([10]), 'Neonate discharged')
    .otherwise('None')
)
#data = data.filter(~col('discharge_disposition_id').isin('Expired', 'Neonate discharged', 'Hospice'))
df = df.withColumn(
    'admission_source_id',
    when(col('admission_source_id').isin([1, 2, 3]), 'Referral')
    .when(col('admission_source_id').isin([4, 5, 6, 10, 18, 19, 25, 26, 22]), 'Transferred from another health care facility')
    .when(col('admission_source_id').isin([9, 15, 17, 20, 21]), 'Not Available')
    .when(col('admission_source_id').isin([11, 12, 13, 14, 23, 24]), 'Delivery')
    .when(col('admission_source_id').isin([7]), 'Emergency')
    .when(col('admission_source_id').isin([8]), 'Court/Law Enforcement')
    .otherwise('None')
)
#data = data.filter(col('admission_source_id') != 'Delivery')

In [394]:
test = test.withColumn(
    'admission_type_id',
    when(col('admission_type_id').isin([1, 2, 7]), 'Emergency')
    .when(col('admission_type_id').isin([5, 6, 8]), 'Not Available')
    .when(col('admission_type_id').isin([3]), 'Elective')
    .when(col('admission_type_id').isin([4]), 'Newborn')
    .otherwise('None')
)
# test = test.filter(col('admission_type_id') != 'Newborn')
test = test.withColumn(
    'discharge_disposition_id',
     when(col('discharge_disposition_id').isin([1]), 'Discharged to home')
    .when(col('discharge_disposition_id').isin([2, 3, 4, 5, 10, 16, 22, 23, 24, 30, 27, 28, 29]), 'Transferred to another medical facility')
    .when(col('discharge_disposition_id').isin([7]), 'Left AMA(Against Medical Advice.)')
    .when(col('discharge_disposition_id').isin([6, 8]), 'Discharged to home with home health service')
    .when(col('discharge_disposition_id').isin([9, 12, 15, 17]), 'Still patient/referred to this institution')
    .when(col('discharge_disposition_id').isin([11, 19, 20, 21]), 'Expired')
    .when(col('discharge_disposition_id').isin([18, 25, 26]), 'Not Available')
    .when(col('discharge_disposition_id').isin([13, 14]), 'Hospice')
    .when(col('discharge_disposition_id').isin([10]), 'Neonate discharged')
    .otherwise('None')
)
# test = test.filter(~col('discharge_disposition_id').isin('Expired', 'Neonate discharged', 'Hospice'))
test = test.withColumn(
    'admission_source_id',
    when(col('admission_source_id').isin([1, 2, 3]), 'Referral')
    .when(col('admission_source_id').isin([4, 5, 6, 10, 18, 19, 25, 26, 22]), 'Transferred from another health care facility')
    .when(col('admission_source_id').isin([9, 15, 17, 20, 21]), 'Not Available')
    .when(col('admission_source_id').isin([11, 12, 13, 14, 23, 24]), 'Delivery')
    .when(col('admission_source_id').isin([7]), 'Emergency')
    .when(col('admission_source_id').isin([8]), 'Court/Law Enforcement')
    .otherwise('None')
)
# test = test.filter(col('admission_source_id') != 'Delivery')

In [395]:
df.groupBy('admission_source_id').count().show()

+--------------------+-----+
| admission_source_id|count|
+--------------------+-----+
|Court/Law Enforce...|   12|
|            Referral|22386|
|Transferred from ...| 5138|
|           Emergency|38844|
|       Not Available| 5133|
|            Delivery|    5|
+--------------------+-----+



In [396]:
df = df.withColumn('admission_source_id', 
                   when((col('admission_source_id') == 'Not Available') & (col('admission_type_id') == 'Emergency'), 'Emergency')
                   .otherwise(col('admission_source_id')))

In [397]:
test = test.withColumn('admission_source_id', 
                   when((col('admission_source_id') == 'Not Available') & (col('admission_type_id') == 'Emergency'), 'Emergency')
                   .otherwise(col('admission_source_id')))

In [398]:
df.filter(col('admission_type_id') == 'Emergency').groupBy('admission_source_id').count().show()

+--------------------+-----+
| admission_source_id|count|
+--------------------+-----+
|Court/Law Enforce...|   10|
|            Referral| 8546|
|Transferred from ...| 4058|
|           Emergency|37625|
|            Delivery|    3|
+--------------------+-----+



In [399]:
df = df.withColumn('admission_source_id', 
                   when((col('admission_source_id') == 'Not Available') & (col('admission_type_id') == 'Elective'), 'Referral')
                   .otherwise(col('admission_source_id')))

In [400]:
test = test.withColumn('admission_source_id', 
                   when((col('admission_source_id') == 'Not Available') & (col('admission_type_id') == 'Elective'), 'Referral')
                   .otherwise(col('admission_source_id')))

In [401]:
df.filter(col('admission_type_id') == 'Elective').groupBy('admission_source_id').count().show()

+--------------------+-----+
| admission_source_id|count|
+--------------------+-----+
|            Referral|12290|
|Transferred from ...| 1011|
|           Emergency|  520|
|Court/Law Enforce...|    1|
|            Delivery|    1|
+--------------------+-----+



In [402]:
df = df.drop('payer_code')

In [403]:
test = test.drop('payer_code')

In [404]:
df = df.drop('medical_specialty')

In [405]:
test = test.drop('medical_specialty')

In [406]:
diag_cols = ['diag_1','diag_2','diag_3']


In [407]:

# Iterate over diagnosis columns
for i in diag_cols:
    df = df.withColumn(
        i,
        when(col(i).startswith('250'), 'Diabetes')
        .when((col(i).cast('int') >= 390) & (col(i).cast('int') < 460) | (col(i) == 785), 'Circulatory')
        .when((col(i).cast('int') >= 460) & (col(i).cast('int') < 520) | (col(i) == 786), 'Respiratory')
        .when((col(i).cast('int') >= 520) & (col(i).cast('int') < 580) | (col(i) == 787), 'Digestive')
        .when((col(i).cast('int') >= 800) & (col(i).cast('int') < 1000), 'Injury and Poisoning')
        .when((col(i).cast('int') >= 710) & (col(i).cast('int') < 740), 'Musculoskeletal System and Connective Tissue')
        .when((col(i).cast('int') >= 580) & (col(i).cast('int') < 630) | (col(i) == 788), 'Genitourinary')
        .when((col(i).cast('int') >= 140) & (col(i).cast('int') < 230), 'Neoplasms')
        .when(col(i).cast('int').isin([780, 781, 784]) | (col(i).cast('int') >= 790) & (col(i).cast('int') < 800), 'Other Symptoms')
        .when((col(i).cast('int') >= 240) & (col(i).cast('int') < 280), 'Endocrine, Nutritional, Metabolic, Immunity')
        .when((col(i).cast('int') >= 680) & (col(i).cast('int') < 710) | (col(i) == 782), 'Skin and Subcutaneous Tissue')
        .when((col(i).cast('int') >= 1) & (col(i).cast('int') < 140), 'Infectious and Parasitic')
        .when((col(i).cast('int') >= 290) & (col(i).cast('int') < 320), 'Mental Disorders')
        .when((col(i).cast('int') >= 280) & (col(i).cast('int') < 290), 'Blood and Blood-Forming Organs')
        .when((col(i).cast('int') >= 320) & (col(i).cast('int') < 360), 'Nervous')
        .when((col(i).cast('int') >= 630) & (col(i).cast('int') < 680), 'Pregnancy, Childbirth')
        .when((col(i).cast('int') >= 360) & (col(i).cast('int') < 390), 'Sense Organs')
        .otherwise('Congenital Anomalies')
    )


In [408]:

# Iterate over diagnosis columns
for i in diag_cols:
    test = test.withColumn(
        i,
        when(col(i).startswith('250'), 'Diabetes')
        .when((col(i).cast('int') >= 390) & (col(i).cast('int') < 460) | (col(i) == 785), 'Circulatory')
        .when((col(i).cast('int') >= 460) & (col(i).cast('int') < 520) | (col(i) == 786), 'Respiratory')
        .when((col(i).cast('int') >= 520) & (col(i).cast('int') < 580) | (col(i) == 787), 'Digestive')
        .when((col(i).cast('int') >= 800) & (col(i).cast('int') < 1000), 'Injury and Poisoning')
        .when((col(i).cast('int') >= 710) & (col(i).cast('int') < 740), 'Musculoskeletal System and Connective Tissue')
        .when((col(i).cast('int') >= 580) & (col(i).cast('int') < 630) | (col(i) == 788), 'Genitourinary')
        .when((col(i).cast('int') >= 140) & (col(i).cast('int') < 230), 'Neoplasms')
        .when(col(i).cast('int').isin([780, 781, 784]) | (col(i).cast('int') >= 790) & (col(i).cast('int') < 800), 'Other Symptoms')
        .when((col(i).cast('int') >= 240) & (col(i).cast('int') < 280), 'Endocrine, Nutritional, Metabolic, Immunity')
        .when((col(i).cast('int') >= 680) & (col(i).cast('int') < 710) | (col(i) == 782), 'Skin and Subcutaneous Tissue')
        .when((col(i).cast('int') >= 1) & (col(i).cast('int') < 140), 'Infectious and Parasitic')
        .when((col(i).cast('int') >= 290) & (col(i).cast('int') < 320), 'Mental Disorders')
        .when((col(i).cast('int') >= 280) & (col(i).cast('int') < 290), 'Blood and Blood-Forming Organs')
        .when((col(i).cast('int') >= 320) & (col(i).cast('int') < 360), 'Nervous')
        .when((col(i).cast('int') >= 630) & (col(i).cast('int') < 680), 'Pregnancy, Childbirth')
        .when((col(i).cast('int') >= 360) & (col(i).cast('int') < 390), 'Sense Organs')
        .otherwise('Congenital Anomalies')
    )


In [409]:
df = df.withColumn(
    'diag_2',
    when((col('diag_2') == 'Missing') & (col('diag_1') != 'Missing'), 'Not Required')
    .otherwise(col('diag_2'))
)
df = df.withColumn(
    'diag_3',
    when((col('diag_3') == 'Missing') & ((col('diag_2') == 'Not Required') | (col('diag_2') != 'Missing')), 'Not Required')
    .otherwise(col('diag_3'))
)

In [410]:
test = test.withColumn(
    'diag_2',
    when((col('diag_2') == 'Missing') & (col('diag_1') != 'Missing'), 'Not Required')
    .otherwise(col('diag_2'))
)
test = test.withColumn(
    'diag_3',
    when((col('diag_3') == 'Missing') & ((col('diag_2') == 'Not Required') | (col('diag_2') != 'Missing')), 'Not Required')
    .otherwise(col('diag_3'))
)

In [411]:
df.groupBy('max_glu_serum').count().show()

+-------------+-----+
|max_glu_serum|count|
+-------------+-----+
|         None|68041|
|         >300|  729|
|         Norm| 1759|
|         >200|  989|
+-------------+-----+



In [412]:
df = df.drop('examide', 'citoglipton','metformin-rosiglitazone')

In [413]:
test = test.drop('examide', 'citoglipton','metformin-rosiglitazone')

In [414]:
drugs = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
         'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
         'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone',
         'metformin-pioglitazone']

# Create temporary columns and calculate 'numchange'
for i in drugs:
    df = df.withColumn(
        i,
        when(col(i) == 'No', -2)
        .when(col(i) == 'Down', -1)
        .when(col(i) == 'Steady', 0)
        .when(col(i) == 'Up', 1)
        .otherwise(col(i))
    )

In [415]:
for i in drugs:
    test = test.withColumn(
        i,
        when(col(i) == 'No', -2)
        .when(col(i) == 'Down', -1)
        .when(col(i) == 'Steady', 0)
        .when(col(i) == 'Up', 1)
        .otherwise(col(i))
    )

In [416]:
for i in drugs:
    df = df.withColumn(i, col(i).cast('integer'))

In [417]:
for i in drugs:
    test = test.withColumn(i, col(i).cast('integer'))

In [418]:
df.groupBy('change').count().show()

+------+-----+
|change|count|
+------+-----+
|    Ch|32008|
|    No|39510|
+------+-----+



In [419]:

# Replace values in 'change' column
df = df.withColumn('change', when(col('change') == 'Ch', 1).otherwise(0))

# Replace values in 'diabetesMed' column
df = df.withColumn('diabetesMed', when(col('diabetesMed') == 'Yes', 1).otherwise(0))

In [420]:

# Replace values in 'change' column
test = test.withColumn('change', when(col('change') == 'Ch', 1).otherwise(0))

# Replace values in 'diabetesMed' column
test = test.withColumn('diabetesMed', when(col('diabetesMed') == 'Yes', 1).otherwise(0))

In [421]:
df = df.withColumn('readmitted', 
                   when(col('readmitted') == '>30', 2)
                   .when(col('readmitted')== '<30', 1)
                    .when(col('readmitted')== 'NO', 0)
                   .otherwise(col('readmitted').cast('int')))

In [422]:
df = df.withColumn(
    'preceding_year_visits',
    col('number_outpatient') + col('number_emergency') + col('number_inpatient')
)

In [423]:
df = df.drop('number_outpatient' ,'number_emergency' ,'number_inpatient')

In [424]:
test = test.withColumn(
    'preceding_year_visits',
    col('number_outpatient') + col('number_emergency') + col('number_inpatient')
)

In [425]:
test = test.drop('number_outpatient' ,'number_emergency' ,'number_inpatient')

In [426]:
df = df.withColumn(
    'number_changes',
    sum(when((col(j) == -1) | (col(j) == 1), 1).otherwise(0) for j in drugs)
)

In [427]:
test = test.withColumn(
    'number_changes',
    sum(when((col(j) == -1) | (col(j) == 1), 1).otherwise(0) for j in drugs)
)

In [428]:
df.groupBy('number_changes').count().show()

+--------------+-----+
|number_changes|count|
+--------------+-----+
|             1|16992|
|             3|   78|
|             2|  902|
|             0|53543|
|             4|    3|
+--------------+-----+



In [429]:
df = df.withColumn(
    'number_diab_meds',
    sum(when(col(j) != -2, 1).otherwise(0) for j in drugs)
)

In [430]:
test = test.withColumn(
    'number_diab_meds',
    sum(when(col(j) != -2, 1).otherwise(0) for j in drugs)
)

In [431]:
df = df.withColumn(
    'insulin_treatment',
    when((col('insulin') != -2) & (col('number_diab_meds') == 1), 'insulin_only')
    .when((col('insulin') != -2) & (col('number_diab_meds') > 1), 'insulin_combo')
    .when((col('insulin') == -2) & (col('number_diab_meds') == 0), 'no_med')
    .otherwise('other_meds')
)


In [432]:
test = test.withColumn(
    'insulin_treatment',
    when((col('insulin') != -2) & (col('number_diab_meds') == 1), 'insulin_only')
    .when((col('insulin') != -2) & (col('number_diab_meds') > 1), 'insulin_combo')
    .when((col('insulin') == -2) & (col('number_diab_meds') == 0), 'no_med')
    .otherwise('other_meds')
)


In [433]:
df.groupBy('insulin_treatment').count().show()

+-----------------+-----+
|insulin_treatment|count|
+-----------------+-----+
|           no_med|17322|
|       other_meds|17013|
|     insulin_only|20807|
|    insulin_combo|16376|
+-----------------+-----+



In [434]:
df = df.drop('number_diab_meds')

In [435]:
test = test.drop('number_diab_meds')

In [436]:
df = df.drop('nateglinide','chlorpropamide','acetohexamide',  'tolbutamide',
              'acarbose', 'miglitol',  'troglitazone',  'tolazamide',
              'glyburide-metformin',  'glipizide-metformin',
              'glimepiride-pioglitazone','metformin-pioglitazone')

In [437]:
test = test.drop('nateglinide','chlorpropamide','acetohexamide',  'tolbutamide',
              'acarbose', 'miglitol',  'troglitazone',  'tolazamide',
              'glyburide-metformin',  'glipizide-metformin',
              'glimepiride-pioglitazone','metformin-pioglitazone')

In [438]:
df = df.drop('change')

In [439]:
test = test.drop('change')

In [440]:
df = df.drop('admission_type_id')

In [441]:
test = test.drop('admission_type_id')

In [442]:
df.columns

['race',
 'gender',
 'age',
 'discharge_disposition_id',
 'admission_source_id',
 'time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'diag_1',
 'diag_2',
 'diag_3',
 'number_diagnoses',
 'max_glu_serum',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'glimepiride',
 'glipizide',
 'glyburide',
 'pioglitazone',
 'rosiglitazone',
 'insulin',
 'diabetesMed',
 'readmitted',
 'preceding_year_visits',
 'number_changes',
 'insulin_treatment']

MODEL


In [443]:
df = df.drop('metformin',	'repaglinide',	'glimepiride',	'glipizide'	,'glyburide',	'pioglitazone',	'rosiglitazone')

In [444]:
test = test.drop('metformin',	'repaglinide',	'glimepiride',	'glipizide'	,'glyburide',	'pioglitazone',	'rosiglitazone')

In [445]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline , PipelineModel

In [446]:
# Index the 'race' column
indexer = StringIndexer(inputCol='race', outputCol='race_index')

# One-hot encode the 'race_index' column
encoder = OneHotEncoder(inputCols=['race_index'], outputCols=['race_encoded'])

# Assemble stages into a pipeline
pipeline = Pipeline(stages=[indexer, encoder])

# Fit and transform the pipeline
final_df = pipeline.fit(df).transform(df)

# Drop the original 'race' and 'race_index' columns
final_df = final_df.drop('race', 'race_index')

# Show the DataFrame after one-hot encoding
#final_df.show()

In [447]:
final_df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- discharge_disposition_id: string (nullable = false)
 |-- admission_source_id: string (nullable = false)
 |-- time_in_hospital: integer (nullable = true)
 |-- num_lab_procedures: integer (nullable = true)
 |-- num_procedures: integer (nullable = true)
 |-- num_medications: integer (nullable = true)
 |-- diag_1: string (nullable = false)
 |-- diag_2: string (nullable = false)
 |-- diag_3: string (nullable = false)
 |-- number_diagnoses: integer (nullable = true)
 |-- max_glu_serum: string (nullable = true)
 |-- A1Cresult: string (nullable = true)
 |-- insulin: integer (nullable = true)
 |-- diabetesMed: integer (nullable = false)
 |-- readmitted: integer (nullable = true)
 |-- preceding_year_visits: integer (nullable = true)
 |-- number_changes: integer (nullable = false)
 |-- insulin_treatment: string (nullable = false)
 |-- race_encoded: vector (nullable = true)



In [448]:
# Index the 'race' column
indexer = StringIndexer(inputCol='race', outputCol='race_index')

# One-hot encode the 'race_index' column
encoder = OneHotEncoder(inputCols=['race_index'], outputCols=['race_encoded'])

# Assemble stages into a pipeline
pipeline = Pipeline(stages=[indexer, encoder])

# Fit and transform the pipeline
final_test = pipeline.fit(test).transform(test)

# Drop the original 'race' and 'race_index' columns
final_test = final_test.drop('race', 'race_index')

# Show the DataFrame after one-hot encoding
#final_test.show()

In [449]:
# List of columns to label encode
columns_to_encode = ['diag_1', 'diag_2', 'diag_3', 'admission_source_id', 'A1Cresult',
                     'insulin_treatment', 'discharge_disposition_id', 'max_glu_serum','diabetesMed','gender']

# Apply StringIndexer to each column
indexers = [StringIndexer(inputCol=column, outputCol=column+"_encoded").fit(final_df) for column in columns_to_encode]

# Apply the StringIndexers in parallel using a Pipeline
pipeline = Pipeline(stages=indexers)
indexed_df = pipeline.fit(final_df).transform(final_df)

# Drop the original columns
final_df = indexed_df.drop(*columns_to_encode)

# Show the DataFrame after label encoding
#final_df.show()

In [450]:
# List of columns to label encode
columns_to_encode = ['diag_1', 'diag_2', 'diag_3', 'admission_source_id', 'A1Cresult',
                     'insulin_treatment', 'discharge_disposition_id', 'max_glu_serum','diabetesMed','gender']

# Apply StringIndexer to each column
indexers = [StringIndexer(inputCol=column, outputCol=column+"_encoded").fit(final_test) for column in columns_to_encode]

# Apply the StringIndexers in parallel using a Pipeline
pipeline = Pipeline(stages=indexers)
indexed_test = pipeline.fit(final_test).transform(final_test)

# Drop the original columns
final_test = indexed_test.drop(*columns_to_encode)

# Show the DataFrame after label encoding
#final_df.show()

In [451]:
# List of columns to consider for dropping rows with missing values
columns_to_check = ['max_glu_serum_encoded', 'A1Cresult_encoded']

# Drop rows with missing values in the specified columns
final_df = final_df.na.drop(subset=columns_to_check)

In [452]:
final_test = final_test.na.drop(subset=columns_to_check)

In [453]:
final_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- time_in_hospital: integer (nullable = true)
 |-- num_lab_procedures: integer (nullable = true)
 |-- num_procedures: integer (nullable = true)
 |-- num_medications: integer (nullable = true)
 |-- number_diagnoses: integer (nullable = true)
 |-- insulin: integer (nullable = true)
 |-- readmitted: integer (nullable = true)
 |-- preceding_year_visits: integer (nullable = true)
 |-- number_changes: integer (nullable = false)
 |-- race_encoded: vector (nullable = true)
 |-- diag_1_encoded: double (nullable = false)
 |-- diag_2_encoded: double (nullable = false)
 |-- diag_3_encoded: double (nullable = false)
 |-- admission_source_id_encoded: double (nullable = false)
 |-- A1Cresult_encoded: double (nullable = false)
 |-- insulin_treatment_encoded: double (nullable = false)
 |-- discharge_disposition_id_encoded: double (nullable = false)
 |-- max_glu_serum_encoded: double (nullable = false)
 |-- diabetesMed_encoded: double (nullable = false)
 |-- g

In [454]:
final_test.printSchema()

root
 |-- index: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- time_in_hospital: integer (nullable = true)
 |-- num_lab_procedures: integer (nullable = true)
 |-- num_procedures: integer (nullable = true)
 |-- num_medications: integer (nullable = true)
 |-- number_diagnoses: integer (nullable = true)
 |-- insulin: integer (nullable = true)
 |-- preceding_year_visits: integer (nullable = true)
 |-- number_changes: integer (nullable = false)
 |-- race_encoded: vector (nullable = true)
 |-- diag_1_encoded: double (nullable = false)
 |-- diag_2_encoded: double (nullable = false)
 |-- diag_3_encoded: double (nullable = false)
 |-- admission_source_id_encoded: double (nullable = false)
 |-- A1Cresult_encoded: double (nullable = false)
 |-- insulin_treatment_encoded: double (nullable = false)
 |-- discharge_disposition_id_encoded: double (nullable = false)
 |-- max_glu_serum_encoded: double (nullable = false)
 |-- diabetesMed_encoded: double (nullable = false)
 |-- gender

In [455]:
data_sample = final_df

In [456]:
data_sample.groupBy('readmitted').count().show()

+----------+-----+
|readmitted|count|
+----------+-----+
|         1| 3224|
|         2|13920|
|         0|54374|
+----------+-----+



In [457]:
class_0 = data_sample.filter(col("readmitted") == 0)
class_1 = data_sample.filter(col("readmitted") == 1)
class_2 = data_sample.filter(col("readmitted") == 2)
class_0_resampled = class_0.sample(True, 1000 / class_0.count(), seed=42)
class_2_resampled = class_2.sample(True, 13920 / class_2.count(), seed=42)
#class_1_resampled = class_1.sample(True, 2000 / class_2.count(), seed=42)
balanced_df = class_1.union(class_0_resampled)
balanced_df = balanced_df.union(class_2_resampled)


In [458]:
balanced_df.groupBy('readmitted').count().show()

+----------+-----+
|readmitted|count|
+----------+-----+
|         1| 3224|
|         0| 1041|
|         2|13982|
+----------+-----+



In [459]:
labelIndex = StringIndexer(inputCol = 'readmitted' , outputCol = 'label' )

In [460]:
indexLabelTrain = labelIndex.fit(balanced_df).transform(balanced_df)

In [461]:
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier,LogisticRegression,NaiveBayes
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler

In [462]:
indexLabelTrain.columns

['age',
 'time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_diagnoses',
 'insulin',
 'readmitted',
 'preceding_year_visits',
 'number_changes',
 'race_encoded',
 'diag_1_encoded',
 'diag_2_encoded',
 'diag_3_encoded',
 'admission_source_id_encoded',
 'A1Cresult_encoded',
 'insulin_treatment_encoded',
 'discharge_disposition_id_encoded',
 'max_glu_serum_encoded',
 'diabetesMed_encoded',
 'gender_encoded',
 'label']

In [463]:
list_index=[]
features_col = ['diag_1', 'diag_2', 'diag_3']
for col_name in features_col:
  list_index.append(col_name+'_encoded')
assembler = VectorAssembler(inputCols=list_index,outputCol="features")

In [464]:
scaler = StandardScaler(inputCol="features", outputCol="features_scaled", withStd=True, withMean=True)


In [465]:
rf_model = RandomForestClassifier(labelCol="label", featuresCol="features_scaled",maxBins = 128,maxDepth = 10, numTrees = 120)

In [466]:
ds_model = DecisionTreeClassifier(labelCol="label", featuresCol="features",maxBins = 128)


In [467]:
pipeline = Pipeline( stages = [assembler, scaler, rf_model])

In [468]:
model = pipeline.fit(indexLabelTrain)

23/12/21 03:18:24 WARN DAGScheduler: Broadcasting large task binary with size 1086.8 KiB
23/12/21 03:18:25 WARN DAGScheduler: Broadcasting large task binary with size 1576.8 KiB
23/12/21 03:18:27 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
23/12/21 03:18:29 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
23/12/21 03:18:33 WARN DAGScheduler: Broadcasting large task binary with size 5.8 MiB
23/12/21 03:18:36 WARN DAGScheduler: Broadcasting large task binary with size 1221.8 KiB
23/12/21 03:18:38 WARN DAGScheduler: Broadcasting large task binary with size 8.6 MiB
23/12/21 03:18:42 WARN DAGScheduler: Broadcasting large task binary with size 1650.1 KiB


In [469]:
result = model.transform(final_test)

In [470]:
prediction = model.transform(indexLabelTrain)

In [471]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(prediction)
print(f"F1 Score: {f1_score}")
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy_score = evaluator.evaluate(prediction)
print(f"Accuracy Score: {accuracy_score}")

23/12/21 03:18:46 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB


F1 Score: 0.6753478340697001


23/12/21 03:18:51 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB


Accuracy Score: 0.7705376226228969


In [472]:
submit_file = result.select('index','prediction')

In [473]:
labelsArray = ["GT30","LT30","NO"]
testSolutions = IndexToString(inputCol="prediction", outputCol="readmitted", labels = labelsArray).transform(submit_file)

In [474]:

final = testSolutions.select('index','readmitted')

In [475]:
final.groupBy('readmitted').count().show()

23/12/21 03:18:56 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB


+----------+-----+
|readmitted|count|
+----------+-----+
|      LT30|  176|
|        NO|    8|
|      GT30|30064|
+----------+-----+



23/12/21 03:18:59 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB


In [476]:
final.toPandas().to_csv("dumpsolutions.csv",header=True, index=False)

23/12/21 03:19:00 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
